# CS492 전산학특강<인공지능 산업 및 스마트에너지>
## Deep Learning Practice 
#### Prof. Ho-Jin Choi
#### School of Computing, KAIST

---

### 2-4. Save and restore models

Model progress can be saved during—and after—training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:
- code to create the model, and
- the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

#### Setup

Install and import TensorFlow and dependencies:

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass

!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.0.0


#### Get an example dataset

To demonstrate how to save and load weights, you'll use the MNIST dataset. To speed up these runs, use the first 1000 examples:

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

#### Define model 
Start by building a simple sequential model:

In [4]:
# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
      ])
    
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    
    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#### Save checkpoints during training (Save weights only)

You can use a _trained model_ without having to retrain it, or pick-up training where you left off—in case the training process was interrupted. The [`tf.keras.callbacks.ModelCheckpoint`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks/ModelCheckpoint) callback allows to continually save the model both during and at the end of training.

##### Checkpoint callback usage
Create a tf.keras.callbacks.ModelCheckpoint callback that **_saves weights only_** during training:

In [5]:
import os

ckpt_path_for_t1 = "checkpoints/training_1/cp.ckpt"
ckpt_dir_for_t1 = os.path.dirname(ckpt_path_for_t1)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_path_for_t1,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 verbose=1)

In [6]:
# Train the model with the new callback
model.fit(train_images, 
         train_labels,
         epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.2287 - accuracy: 0.6510 
Epoch 00001: val_loss improved from inf to 0.73770, saving model to checkpoints/training_1/cp.ckpt
1000/1000 [==============================] - 1s 862us/sample - loss: 1.2028 - accuracy: 0.6600 - val_loss: 0.7377 - val_accuracy: 0.7800
Epoch 2/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.4473 - accuracy: 0.8831
Epoch 00002: val_loss improved from 0.73770 to 0.51823, saving model to checkpoints/training_1/cp.ckpt
1000/1000 [==============================] - 0s 132us/sample - loss: 0.4446 - accuracy: 0.8840 - val_loss: 0.5182 - val_accuracy: 0.8480
Epoch 3/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2892 - accuracy: 0.9365
Epoch 00003: val_loss improved from 0.51823 to 0.46523, saving model to checkpoints/training_1/cp.ckpt
1000/1000 [==============================] - 0s 128us/sample - loss: 0.2856 - 

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [7]:
!ls {ckpt_dir_for_t1}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


Create a new, untrained model. When restoring a model from weights-only, you **must have a model with the same architecture as the original model.** Since it's the same model architecture, **you can share weights** despite that it's a different instance of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [8]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.4137 - accuracy: 0.1410
Untrained model, accuracy: 14.10%


Then load the weights from the checkpoint and re-evaluate:

In [9]:
# Loads the weights
model.load_weights(ckpt_path_for_t1)

# Re-evaluate the model
loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4848 - accuracy: 0.8710
Restored model, accuracy: 87.10%


##### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency. 

Train a new model, and **save uniquely named checkpoints once every five epochs**:

In [14]:
# Include the epoch in the file name (uses `str.format`)
ckpt_path_for_t2 = "checkpoints/training_2/cp-{epoch:04d}.ckpt"
ckpt_dir_for_t2 = os.path.dirname(ckpt_path_for_t2)

if not os.path.exists(ckpt_dir_for_t2):
    os.makedirs(ckpt_dir_for_t2)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_path_for_t2, 
                                                 verbose=1, 
                                                 save_weights_only=True,
                                                 period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(ckpt_dir_for_t2.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
         train_labels, 
         callbacks=[cp_callback],
         validation_data=(test_images, test_labels),
         verbose=0)

W0316 07:46:26.066195 140158177752832 callbacks.py:863] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Now, look at the resulting checkpoints and choose the latest one:

In [ ]:
! ls {ckpt_dir_for_t2}

##### What are these files?
The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain: * One or more shards that contain your model's weights. * An index file that indicates which weights are stored in a which shard.

If you are only training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

In [ ]:
latest = tf.train.latest_checkpoint(ckpt_dir_for_t2)
latest

To test, reset the model and load the latest checkpoint:

In [ ]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels,verbose=0)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

##### Manually save weights

You saw how to load the weights into a model. Manually saving them is just as simple with the `Model.save_weights` method. By default, tf.keras—and `save_weights` in particular—uses the TensorFlow checkpoints format with a .ckpt extension

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint/cur_weights')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint/cur_weights')

# Evaluate the model
loss,acc = model.evaluate(test_images, test_labels, verbose=0)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

#### Save the entire model 

The model and optimizer can be saved to a file that contains both their _state (weights and variables)_ and the _model configuration_. This allows you to export a model so it can be used without access to the original Python code. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([HDF5 in TensorFlow.js](https://www.tensorflow.org/js/tutorials/conversion/import_keras), [Saved Model in TensorFlow.js](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([HDF5 in Python](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model in Python](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))

##### Save model as an HDF5 file

Keras also provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. For our purposes, the saved model can be treated as a single binary blob:

In [ ]:
# Create a new model instance
model = create_model()

# Train the model
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file
save_dir = 'saved_models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

model.save(os.path.join(save_dir, 'my_model.h5'))

Now, recreate the model from that file:

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = 

# Show the model architecture
new_model.summary()

Check its accuracy:

In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=0)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

This technique saves everything:

- The weight values
- The model's configuration(architecture)
- The optimizer configuration

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from tf.train). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.

##### As a saved_model

Build a new model, then train it:

In [ ]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Create a saved_model, and place it in a time-stamped directory with `tf.keras.experimental.export_saved_model`:

In [ ]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

In [ ]:
!ls saved_models/

In [ ]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)

# Check its architecture
new_model.summary()

In [ ]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=model.optimizer,  # Keep the optimizer that was loaded
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=0)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
gks8030@gmail.com 

/ seunghohan@kaist.ac.kr

KLMS

010-8030-6802

